In [1]:
import tensorflow as tf
import numpy as np
import cv2
import gc
import random

import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D, Activation, AveragePooling2D, ReLU, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import regularizers, backend, Sequential, optimizers, metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json, load_model, Model
from tensorflow.keras.callbacks import TensorBoard, Callback, ReduceLROnPlateau
from tensorflow.keras.initializers import he_normal


In [2]:
import tensorflow.keras.applications.resnet_v2 as resnet_v2


def history_show(history):
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(history.history['loss'], 'y', label='train loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='best')

    acc_ax.plot(history.history['acc'], 'b', label='train acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='best')
    
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
batch_size = 32
epochs = 32
verbose = 1
lr = 1e-2
data_dir = 'nipa_dataset/train'
preprocess_input = resnet_v2.preprocess_input

seed = random.randrange(0, 50000)
print("random seed : ", seed)


datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

datagen_test = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)


#reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.3, verbose=1, patience=max_step)
 
train_generator = datagen.flow_from_directory(directory=data_dir, target_size=(256,256), batch_size=batch_size,
                                       interpolation='bicubic', subset='training', seed=seed)    
val_generator = datagen_test.flow_from_directory(directory=data_dir, target_size=(256,256), batch_size=batch_size,
                                       interpolation='bicubic', subset='validation', seed=seed)     

model = resnet_v2.ResNet101V2(include_top=True, weights=None, input_shape=(256,256,3), classes=20)
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr), 
              metrics=['accuracy', f1_m, precision_m, recall_m])
    
model.summary()

print("training")

history = model.fit_generator(generator=train_generator, epochs=epochs, validation_data=val_generator, 
                          verbose=verbose)

history_show(history)

print("model validation")
loss, accuracy, f1_score, precision, recall = model.evaluate_generator(generator=val_generator, verbose=verbose)
print("loss : %f   accuracy : %f    f1 : %f    precision : %f    recall : %f" % (loss, accuracy, f1_score, precision, recall))

model.save("nipa_pretest_resnet101.h5")

random seed :  47332
Found 12800 images belonging to 20 classes.
Found 3200 images belonging to 20 classes.
Model: "resnet101v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 130, 130, 64) 0           conv1_conv[0][

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 100 steps
Epoch 1/32
400/400 [==============================] - 195s 486ms/step - loss: 2.4011 - accuracy: 0.2535 - f1_m: 0.1268 - precision_m: 0.6102 - recall_m: 0.0755 - val_loss: 2.5539 - val_accuracy: 0.3281 - val_f1_m: 0.2000 - val_precision_m: 0.5073 - val_recall_m: 0.1262
Epoch 2/32
400/400 [==============================] - 181s 452ms/step - loss: 1.5000 - accuracy: 0.5191 - f1_m: 0.4469 - precision_m: 0.7527 - recall_m: 0.3277 - val_loss: 1.7388 - val_accuracy: 0.4884 - val_f1_m: 0.4754 - val_precision_m: 0.6392 - val_recall_m: 0.3816
Epoch 3/32
400/400 [==============================] - 189s 474ms/step - loss: 1.0751 - accuracy: 0.6529 - f1_m: 0.6269 - precision_m: 0.7926 - recall_m: 0.5238 - val_loss: 1.2753 - val_accuracy: 0.6134 - val_f1_m: 0.6125 - val_precision_m: 0.7313 - val_recall_m: 0.5291
Epoch 4/32
400/400 [==============================] - 190s 474ms/step - loss: 0.7777 - accuracy: